## Replace the dev_id, auth_key, and player_name with you developer id, authentication id (from HiRez), and your in-game player name.

In [ ]:
import pyrez
import numpy as np

dev_id = ''
auth_key = ''
player_name = ''
paladins = pyrez.PaladinsAPI(dev_id, auth_key)

## Gets a backlog of match data by pulling match ids from paladins.guru. The paladins API only gives the most recent 50 matches for the player and this is a way to get more. Only run to get the first set of matches, otherwise use the code for updating with recent matches.

In [ ]:
from lxml import html
import requests
import re
import pickle

player_id = paladins.getPlayer(player_name)['ActivePlayerId']
id_list = []
for j in range(1, 13):
    page = requests.get('https://paladins.guru/profile/' + str(player_id) + '-' + player_name + '/matches?page=' + str(j))
    indexes = [m.start() for m in re.finditer('/match/', page.content.decode("utf-8"))]

    for i in indexes:
        id_list.append(page.content.decode("utf-8")[i+7:i+16])
        
match_list = []
for j in id_list:
    match = paladins.getMatch(j)
    if len(match) != 0:
        match_list.append(match)
        
with open('match_list.pkl', 'wb') as f:
    pickle.dump(match_list, f)
f.close()

## Updates the match list with recent matches. It is recommended to make a copy of match_list.py after running to avoid it being overwritten from accidentally running the code to get matches from paladins.guru.

In [ ]:
new_matches = paladins.getMatchHistory(player_name)

import pickle
import numpy as np

with open('match_list.pkl', 'rb') as f:
    match_list = pickle.load(f)
f.close()

match_ids = []
for match in match_list:
    match_ids.append(match[0]['Match'])

new_match_ids = []
for match in new_matches:
    if match['Match'] not in match_ids and 'Training' not in match['Queue']:
        new_match_ids.append(match['Match'])
        
print(len(new_match_ids), len(match_list))

for j in new_match_ids:
    match = paladins.getMatch(j)
    if len(match) != 0:
        match_list.append(match)
        
print(len(match_list))

with open('match_list.pkl', 'wb') as f:
    pickle.dump(match_list, f)
f.close()